# Safety Hardhat Detection in Construction: Deep Learning Tutorial
**Author:** Ruoxin Xiong, Ph.D., Assistant Professor \\
**Affiliation:** Construction Management, College of Architecture and Environmental Design, Kent State University

## Overview
In this notebook, you will:
1. Introduce the *Hard Hat Detection* dataset from Kaggle.
2. Load and split the dataset into training, validation, and testing sets.
3. Set up a [**YOLOv5** (You Only Look Once, version 5)](https://docs.ultralytics.com/) environment to train an object detection model on the dataset.
4. Evaluate the model's performance using common metrics (mAP, precision, recall).
5. Provide tips for next steps and improvements.

> **Note:** This notebook is designed for Google Colab with GPU enabled. \
 Make sure to **enable GPU** under `*Runtime* > *Change runtime type* > *Hardware Accelerator* = GPU`.

# **Safety Hardhat Detection in Construction**

### Why Hardhat Detection?
- Construction sites can be **dangerous** if workers do not follow proper safety protocols (like wearing a helmet / hardhat).
- Automated **computer vision** can detect if workers in images or video feeds are wearing helmets.
- Useful for **real-time safety monitoring**, compliance reporting, or risk management.

### About the Dataset
- *Hard Hat Detection Dataset* ([link](https://www.kaggle.com/datasets/andrewmvd/hard-hat-detection)) has ~10,000 images.
- Annotations include bounding boxes for:
  1. `helmet`
  2. `head` (no helmet)
- We'll focus on detecting whether a worker's head is protected by a helmet or not.

### Tools We'll Use
1. **Python** + **pandas**, **matplotlib** for data handling and visualization.
2. **YOLOv5** for object detection:
   - YOLO is a popular, state-of-the-art object detection approach.
   - We can quickly train a custom model on new data.
3. **Google Colab** or local GPU environment for faster training (recommended, but you can do CPU-only with slower training).

## 1. Downloading the Dataset

We use `gdown` to download the zipped dataset from a shared Google Drive link. Replace `<YOUR_GOOGLE_DRIVE_FILE_ID>` with your file's ID. The expected folder structure after unzipping is:

```
HardHat_Dataset/
   ├── images/
   │    ├── image_0001.png # All image files (.png)
   │    ├── ...
   └── annotations/
        ├── image_0001.xml # Annotation files in VOC XML format
        ├── ...
```

In [ ]:
# Install gdown if not already installed
!pip install gdown

### Download the Dataset from Google Drive
Replace `<YOUR_GOOGLE_DRIVE_FILE_ID>` with your shared file ID. This file should be a zip archive of the dataset.

- **Full-size dataset** (`HardHat_Dataset`) contains ~10,000 images for model training and testing. Link: https://drive.google.com/file/d/1ZDGJ3tWMqRAdbHviFYXHyvjJQ88tL_c5/view
- **Small-size dataset** (`HardHat_Dataset_1k`) contains ~1,000 images randomly sampled from the full dataset. Link: https://drive.google.com/file/d/1XebIf0c3LDe_KNcPR6KI8Ys4ReNj6knG/view

In [ ]:
import gdown

# If you are using full datset, please replace with your file ID from the shared Google Drive link
file_id = '1XebIf0c3LDe_KNcPR6KI8Ys4ReNj6knG'
url = f'https://drive.google.com/uc?id={file_id}'
output = '/content/hardhat_dataset_1k.zip'

gdown.download(url, output, quiet=False)

# Unzip the dataset
!unzip -q /content/hardhat_dataset_1k.zip -d /content/HardHat_Dataset_1k
print("Dataset downloaded and unzipped to /content/HardHat_Dataset_1k")

## 2. Split the Dataset into Train, Validation, and Test Sets

The raw dataset does not have predefined splits. We'll create three subsets (80% train, 10% validation, 10% test). Additionally, because YOLOv5 expects annotation files in a folder named `labels` (in YOLO format), we'll convert the XML annotations (VOC format) to YOLO format during the splitting process.

The expected folder structure after splitting:

```
HardHat_Dataset/
   ├── images/           (raw images)
   ├── annotations/      (raw XML annotations)
   ├── train/
   │    ├── images/
   │    └── labels/     (converted YOLO annotations)
   ├── val/
   │    ├── images/
   │    └── labels/
   └── test/
        ├── images/
        └── labels/
```

In [ ]:
import os, shutil, random
import xml.etree.ElementTree as ET

# Set seed for reproducibility
random.seed(42)

# Define raw dataset directories
base_dir = '/content/HardHat_Dataset_1k'
orig_images_dir = os.path.join(base_dir, 'images')
orig_ann_dir = os.path.join(base_dir, 'annotations')

# Create new split directories for train, val, test (with 'images' and 'labels' folders)
splits = ['train', 'val', 'test']
for split in splits:
    os.makedirs(os.path.join(base_dir, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, split, 'labels'), exist_ok=True)

# Get list of all image files
all_images = [f for f in os.listdir(orig_images_dir) if f.endswith('.png')]
total_images = len(all_images)
print(f"Total images found: {total_images}")

# Shuffle and split images into 80% train, 10% val, 10% test
random.shuffle(all_images)
train_end = int(0.8 * total_images)
val_end = int(0.9 * total_images)

train_imgs = all_images[:train_end]
val_imgs = all_images[train_end:val_end]
test_imgs = all_images[val_end:]

print(f"Train: {len(train_imgs)}, Validation: {len(val_imgs)}, Test: {len(test_imgs)}")

In [ ]:
# Define the classes for YOLO conversion (adjust if needed)
classes = ['helmet', 'head']

def convert_xml_to_yolo(xml_file, classes):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    size = root.find('size')
    width = float(size.find('width').text)
    height = float(size.find('height').text)
    yolo_lines = []
    for obj in root.findall('object'):
        cls = obj.find('name').text
        if cls not in classes:
            continue
        cls_id = classes.index(cls)
        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)
        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        bbox_width = (xmax - xmin) / width
        bbox_height = (ymax - ymin) / height
        yolo_lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")
    return yolo_lines

def copy_and_convert_files(file_list, src_images, src_ann, dest_split):
    dest_img = os.path.join(base_dir, dest_split, 'images')
    dest_lbl = os.path.join(base_dir, dest_split, 'labels')
    for fname in file_list:
        # Copy image file
        shutil.copy(os.path.join(src_images, fname), dest_img)

        base_name = os.path.splitext(fname)[0]
        xml_file = os.path.join(src_ann, base_name + '.xml')
        txt_file = os.path.join(src_ann, base_name + '.txt')

        # If annotation is in XML, convert it; otherwise copy txt
        if os.path.exists(xml_file):
            yolo_lines = convert_xml_to_yolo(xml_file, classes)
            if yolo_lines:
                with open(os.path.join(dest_lbl, base_name + '.txt'), 'w') as f:
                    f.write("\n".join(yolo_lines))
        elif os.path.exists(txt_file):
            shutil.copy(txt_file, dest_lbl)
        else:
            print(f"Warning: No annotation found for {fname}")

# Copy and convert annotations for each split
copy_and_convert_files(train_imgs, orig_images_dir, orig_ann_dir, 'train')
copy_and_convert_files(val_imgs, orig_images_dir, orig_ann_dir, 'val')
copy_and_convert_files(test_imgs, orig_images_dir, orig_ann_dir, 'test')

print("Dataset split and annotations converted (if needed) into train, validation, and test sets.")

In [ ]:
# Optional: Verify splits
for split in splits:
    images_count = len(os.listdir(os.path.join(base_dir, split, 'images')))
    labels_count = len(os.listdir(os.path.join(base_dir, split, 'labels')))
    print(f"{split.capitalize()} - Images: {images_count}, Labels: {labels_count}")

## 3. Basic Data Exploration & Visualization
Let's do a quick check on the **number of images** and show some **sample bounding boxes**.

In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt
import os

image_files = glob.glob(os.path.join(base_dir, 'images', '*.png'))
print("Total Images:", len(image_files))

# Display a random sample image (without bounding boxes for now)
import random
seed = 0

sample_img = random.choice(image_files)
img_bgr = cv2.imread(sample_img)  # BGR format
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(6,6))
plt.imshow(img_rgb)
plt.title(os.path.basename(sample_img))
plt.axis('off')
plt.show()

### Display bounding boxes for the random image

In [ ]:
# Define a function to parse VOC XML and return bounding boxes and class names
def parse_voc_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    boxes = []
    for obj in root.findall('object'):
        cls = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = int(float(bndbox.find('xmin').text))
        ymin = int(float(bndbox.find('ymin').text))
        xmax = int(float(bndbox.find('xmax').text))
        ymax = int(float(bndbox.find('ymax').text))
        boxes.append((cls, xmin, ymin, xmax, ymax))
    return boxes

# Get the corresponding annotation file (assumes same basename with .xml extension)
base_name = os.path.splitext(os.path.basename(sample_img))[0]
xml_file = os.path.join(base_dir, 'annotations', base_name + '.xml')

# Parse XML annotations if available
if os.path.exists(xml_file):
    boxes = parse_voc_xml(xml_file)
else:
    boxes = []
    print("Annotation XML not found for", sample_img)

# Draw bounding boxes on the image
for (cls, xmin, ymin, xmax, ymax) in boxes:
    # Draw rectangle: color (0, 255, 0) is green and thickness 2
    cv2.rectangle(img_rgb, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    # Put class text on top-left corner of the bounding box
    cv2.putText(img_rgb, cls, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX,
                0.9, (0, 255, 0), 2)

# Display the image with bounding boxes
plt.figure(figsize=(6,6))
plt.imshow(img_rgb)
plt.title(os.path.basename(sample_img))
plt.axis('off')
plt.show()

## 4. YOLOv5 Dataset Configuration

Create a YAML configuration file (named `hardhat.yaml`) for YOLOv5.

YOLOv5 expects a directory structure like:
```
yolov5/
   ├── data/
   │    └── your_dataset.yaml   (dataset config)
   ├── dataset images
   └── dataset labels
```
We'll create a **configuration file** that points to your train/test images and the class names. For example:
```
hardhat.yaml:
train: /content/HardHat_Dataset_1k/train/images
val: /content/HardHat_Dataset_1k/val/images
test: /content/HardHat_Dataset_1k/test/images  # optional

names: ["helmet", "head"]  # or your set of classes
nc: 2  # number of classes
```

In [ ]:
import yaml

config_data = {
    'train': os.path.join(base_dir, 'train', 'images'),
    'val': os.path.join(base_dir, 'val', 'images'),
    'names': ['helmet', 'head'],
    'nc': 2
}

with open('hardhat.yaml', 'w') as f:
    yaml.dump(config_data, f)

print("Created YOLOv5 dataset configuration file: hardhat.yaml")

## 5. Setting Up YOLOv5 Environment

If you haven't already cloned YOLOv5, do so now. This code is intended to run in Google Colab.

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
print("Running in Colab?", IN_COLAB)

if IN_COLAB:
    !git clone https://github.com/ultralytics/yolov5.git
    %cd yolov5
    !pip install -r requirements.txt
else:
    print("Ensure YOLOv5 is installed locally.")

## 6. Training YOLOv5 on the Dataset

Now that the dataset is split and the configuration file is ready, we can start training.

From within the `yolov5/` directory, you can run:
```
!python train.py --img 640 --batch 16 --epochs 30 \
  --data hardhat.yaml --weights yolov5s.pt \
  --name yolo_hardhat_exp
```

Explaining each argument:
- **--img 640**: The image resolution for training.
- **--batch 16**: Batch size (adjust based on GPU memory).
- **--epochs 10**: Number of training epochs. Increase if you have enough time and data.
- **--data hardhat.yaml**: Path to your dataset config file.
- **--weights yolov5s.pt**: Starting from a pretrained YOLOv5 model.
- **--name yolo_hardhat_exp**: Output folder name for results.

During training, YOLOv5 will display training/validation losses, mAP (mean Average Precision), and more.

- **mAP@0.5**: The primary object detection metric. Closer to 1 means better.
- **Precision / Recall**: Also measured for each class. Good to see whether the model is catching heads with and without helmets.

During training, YOLOv5 logs metrics per epoch. After training finishes, you can look at `runs/train/yolo_hardhat_exp` for:
- `results.png` plot of training/validation curves.
- Best weights stored as `best.pt`.

In [ ]:
if IN_COLAB:
    %cd /content/yolov5
    !python train.py --img 640 --batch 16 --epochs 10 --data ../hardhat.yaml --weights yolov5s.pt --name yolo_hardhat_exp
else:
    print("Run the YOLOv5 training command in your local environment.")

## 7. Model Testing & Performance Evaluation on Test Subset
Use your trained model to predict on **unseen** images:
```
!python detect.py --weights runs/train/yolo_hardhat_exp/weights/best.pt \
                  --img 640 --conf 0.25 --source /content/HardHat_Dataset-1k/test/images
```
This will create bounding box predictions in `runs/detect/`.


In [ ]:
if 'google.colab' in sys.modules:
    %cd /content/yolov5
    print("Running inference on test set with --save-txt...")
    !python detect.py --weights runs/train/yolo_hardhat_exp/weights/best.pt --img 640 --conf 0.25 --save-txt --source ../HardHat_Dataset_1k/test/images --name test_inference
else:
    print("Run the detection command in your local environment with --save-txt.")

## 8. Visualizing Inference Results

After running inference, check the folder `runs/detect/test_inference`  folder for images with bounding boxes over `helmet` or `head`.
A typical bounding box label might read `helmet 0.91`, indicating the model is **91% confident** it's a helmet.

In [ ]:
from IPython.display import Image, display

# Set path to a sample result image (adjust filename if necessary)
result_img = '/content/yolov5/runs/detect/test_inference/hard_hat_workers1046.png'
if os.path.exists(result_img):
    display(Image(filename=result_img))
else:
    print("Result image not found. Check your detection output folder.")

### Display 5 detected result images with bounding boxes

In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt
from IPython.display import display, Image

# Define the directory where YOLOv5 saves detected result images (with bounding boxes)
# This directory is created by the detect.py script when run with the --save-txt flag.
detected_results_dir = '/content/yolov5/runs/detect/test_inference'

# Gather list of detected result image files
detected_image_files = glob.glob(f"{detected_results_dir}/*.png")
print(f"Found {len(detected_image_files)} detected result images.")

# Display 5 detected result images with bounding boxes
for img_path in detected_image_files[:5]:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6,6))
    plt.imshow(img)
    plt.title(f"Detected Result: {os.path.basename(img_path)}")
    plt.axis('off')
    plt.show()

## 9. Tips for Improvement & Next Steps

- **Edge Deployment**: Running these models on site (in real-time) may require smaller, faster models or specialized hardware.
- **False Positives/Negatives**: A missed detection of a worker without a helmet can have safety implications.
- **Privacy & Ethics**: Worker monitoring must follow local regulations and respect privacy.
- **Integration**: Alerts or logs can integrate with a construction management system.


## 10. Conclusion & Next Steps

In this notebook, you've:
1. Explored how to set up YOLOv5 for **hardhat detection**.
2. Learned basic steps of data preparation, training, and inference.
3. Seen how to interpret object detection metrics (mAP, precision, recall).

**Next Steps**:
- Expand your dataset or gather your own site images.
- Tune hyperparameters, try advanced YOLO versions or other detection frameworks.
- Explore **live camera feed** integration if you want real-time detection on a construction site.
- Keep refining the model, especially for edge cases (nighttime, partial occlusions, reflective surfaces).

Deep learning can **dramatically** improve safety monitoring and compliance tracking for construction teams. Continue learning, stay curious, and best of luck in building a safer job site with AI!

---
# **Resources & References**
1. [YOLOv5 GitHub](https://github.com/ultralytics/yolov5)
2. [Kaggle Hard Hat Detection](https://www.kaggle.com/datasets/andrewmvd/hard-hat-detection)
3. [Ultralytics Documentation](https://docs.ultralytics.com/) for YOLOv5 usage.
4. [Albumentations Library](https://github.com/albumentations-team/albumentations) for data augmentation.
5. Additional frameworks: [Detectron2 (Facebook AI)](https://github.com/facebookresearch/detectron2), [MMDetection](https://github.com/open-mmlab/mmdetection).

Feel free to modify paths, hyperparameters, or configurations as needed.